# Introduction au traitement de texte

Vous avez beaucoup travaillé sur le traitement d'image dans les TPs précédents, on va maintenant s'intéresser au traitement de texte. Dans ce TP on va se concentrer sur de la classification de texte. Vous verrez dans les prochains TPs comment générer du texte, comment faire de la traduction automatique, etc.

Pour ce TP, on va utiliser le dataset `AG_NEWS`, qui contient des articles de journaux classés en 4 catégories : World, Sports, Business, Sci/Tech.

On va tout d'abord télécharger le dataset et voir comment il ressemble.



## Téléchargement du dataset

Avant de télécharger le dataset, on va d'abord installer les librairies `torchtext` et `portalocker`.

Si vous êtes sur colaboratory, vous pouvez ajouter une cellule de code et taper `!pip install torchtext portalocker` pour installer les librairies.

In [1]:
import torch
from torchtext.datasets import AG_NEWS

train_dataset, test_dataset = AG_NEWS(root='./data')

print(type(train_dataset))

<class 'torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe'>


## Exercice 1: Affichage du dataset
Essayez d'affichier quelques articles du dataset, et les labels associés. `train_dataset` est un itérable, donc vous pouvez itérer dessus avec une boucle `for`

In [ ]:
for label, text in train_dataset:
	print(label, text)

## L'idée de la méthode pour classifier du texte

Tout d'abord, on va split le texte en une liste de mots. Ensuite, on va représenter chaque mot par un vecteur. Puis on va faire la moyenne de tous les vecteurs de la phrase pour obtenir un vecteur de taille fixe qui représente le texte qu'on pourrait le faire passer dans un réseau de neurones pour classifier le texte.

Pourquoi faire la moyenne de tous les vecteurs?

- Parce que si on a une phrase avec 3 mots, on va avoir 3 vecteurs pour représenter cette phrase. Si on a une autre phrase avec 5 mots, on va en avoir 5 pour représenter la phrase.
Donc si on veut faire passer ces phrases dans un réseau de neurones, on va avoir des vecteurs de taille différente en entrée, et ça ne va pas marcher car les couches linéaires veulent des vecteurs de taille fixe en entrée. (Si vous vous souvenez bien, il faut préciser `input_dim` pour `nn.Linear`).


Mais comment représenter un mot par un vecteur? Et plus précisément par quel vecteur? Comment on peut déterminer les valeurs de ces vecteurs?
- Ça va être difficile de déterminer les valeurs de ces vecteurs par nous-même, mais on peut les apprendre! On va initialiser les vecteurs avec des valeurs aléatoires, et on va les modifier pendant l'entraînement de notre réseau de neurones qui classifie le texte avec la backprop.

Donc chaque mot va être représenté par un vecteur de poids de taille `D`, qui est initialisé aléatoirement. Et on va avoir `V` mots dans notre vocabulaire. On va donc avoir une matrice de taille `VxD` qui va contenir les vecteurs de tous les mots de notre vocabulaire. Ces vecteurs s'appellent d'ailleurs des `embeddings`.

On pourrait créer manuellement cette matrice, mais il existe une couche dans PyTorch qui va le faire pour nous : `nn.Embedding`. Cette couche va prendre en entrée un entier, et va retourner le vecteur associé à cet entier. Cette couche va aussi apprendre les vecteurs pendant l'entraînement.

Ici, ces entiers dans notre cas vont correspondre à l'index du mot dans le vocabulaire. On va donc devoir associer un index à chaque mot de notre vocabulaire.
Tout d'abord, on va utiliser un tokenizer, qui va prendre en entrée une phrase, et va retourner une liste de mots. L'avantage d'un tokenizer c'est qu'il va aussi enlever les majuscules, les caractères spéciaux, etc. On va utiliser le tokenizer de `torchtext`, qui est un tokenizer très simple.
On va ensuite créer un vocabulaire, qui va associer un index à chaque mot. On va ensuite utiliser ce vocabulaire pour transformer chaque mot en un index, et on va utiliser la couche `nn.Embedding` pour transformer chaque index en un vecteur.

Donc pour résumer:
- On va créer un tokenizer et un vocabulaire, pour associer un index à chaque mot
- On va créer une couche `nn.Embedding` qui va prendre en entrée un index, et retourner le vecteur associé à cet index
- On va créer un réseau de neurones, qui va prendre en entrée la moyenne des vecteurs de tous les mots de la phrase, et qui va retourner la classe du texte.
- On va entraîner le réseau de neurones, et les vecteurs de la couche `nn.Embedding` vont être appris pendant l'entraînement.

![Text Classify](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/TP7_Intro_NLP/textclassify.png)

## Création du tokenizer et vocabulaire
On va utiliser `get_tokenizer` pour créer un tokenizer. Vous pouvez avoir plus d'informations sur cette fonction ici : https://pytorch.org/text/stable/data_utils.html

In [25]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('basic_english')

tokenizer("Hello woRld!")

['hello', 'world', '!']

On va maintenant créer un vocabulaire, qui va associer un index à chaque mot.

On va utiliser `build_vocab_from_iterator` pour créer un vocabulaire à partir d'un itérateur. Vous pouvez avoir plus d'informations sur cette fonction ici : https://pytorch.org/text/stable/vocab.html#torchtext.vocab.Vocab

Un itérateur est un objet qui va nous permettre d'itérer sur le dataset (avec une boucle for par exemple), comme le train_dataset, qui est un itérateur sur le dataset d'entraînement où chaque élément est un tuple `(label, text)`.

Or, pour notre vocabulaire, on a besoin d'un itérateur sur les textes tokenisés, donc on va créer un itérateur `yield_tokens(train_dataset)` où chaque élément est un texte tokenisé du dataset d'entraînement.
Pour ça, on va créer une fonction `yield_tokens` qui va prendre en entrée un itérateur sur le dataset, et qui va retourner un itérateur sur les textes tokenisés du dataset.

In [13]:
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(data_iter):
	for _, text in data_iter:
		yield tokenizer(text)

vocab = build_vocab_from_iterator(iterator=yield_tokens(train_dataset), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

On peut maintenant utiliser le vocabulaire pour transformer un texte en une liste d'index.

In [17]:
vocab(tokenizer("Hello woRld"))

[12544, 50]

### Remarque
`yield_tokens` nous donne un itérateur sur les textes tokenisés donc on peut itérer dessus avec une boucle for. Exemple:

In [19]:
for text in yield_tokens(train_dataset):
	print(text)
	break

['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short-sellers', ',', 'wall', 'street', "'", 's', 'dwindling\\band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.']


### Remarque
L'argument `specials` de `build_vocab_from_iterator` permet de spécifier des tokens spéciaux, qui ne sont pas dans le dataset, mais qu'on veut quand même ajouter au vocabulaire. Ici, on ajoute le token `<unk>` qui va être utilisé pour les mots qui ne sont pas dans le vocabulaire et le token `<pad>` qui va être utilisé pour le padding, qu'on verra plus détaillé plus tard.
`vocab.set_default_index(vocab["<unk>"])` permet de spécifier que l'index par défaut est l'index du token `<unk>`. Donc si on demande l'index d'un mot qui n'est pas dans le vocabulaire, on va avoir l'index du token `<unk>`.

In [21]:
print(vocab["<unk>"])
print(vocab["asdklfjaksljsdaf"])
print(vocab["hello"])

0
0
12544


Je sais que tout ça n'est pas encore très clair, mais vous allez pouvoir vous entraîner sur le challenge après ce TP.

## Préparation du dataset d'entraînement
Tout d'abord, on va convertir notre `train_dataset` en un dataset de liste d'index de mots.

In [28]:
X_train = []
Y_train = []
for label, text in train_dataset:
	X_train.append(vocab(tokenizer(text)))
	Y_train.append(label - 1) # remarque, les labels sont entre 1 et 4, mais on veut qu'ils soient entre 0 et 3

print(X_train[0])
print(Y_train[0])

print(X_train[1])
print(Y_train[1])

[431, 425, 1, 1605, 14838, 113, 66, 2, 848, 13, 27, 14, 27, 15, 50725, 3, 431, 374, 16, 9, 67507, 6, 52258, 3, 42, 4009, 783, 325, 1]
2
[15874, 1072, 854, 1310, 4250, 13, 27, 14, 27, 15, 929, 797, 320, 15874, 98, 3, 27657, 28, 5, 4459, 11, 564, 52790, 8, 80617, 2125, 7, 2, 525, 241, 3, 28, 3890, 82814, 6574, 10, 206, 359, 6, 2, 126, 1]
2


Maintenant, on aimerait convertir X_train en un `torch.tensor`, mais on a un problème : les phrases n'ont pas toutes la même longueur, donc on ne peut pas créer un `torch.tensor` avec toutes les phrases. On va donc devoir "padder" les phrases, c'est-à-dire ajouter des "\<pad>" à la fin des phrases pour qu'elles aient toutes la même longueur. On va donc calculer la longueur maximale des phrases et mettre du padding à la fin des phrases qui sont plus courtes que la longueur maximale.

### Exercice:
Mettre des `vocab["<pad>"]` à la fin de chaque élément de `X_train` pour que tous les éléments de `X_train` aient la même longueur.

In [32]:
...

print(X_train[0])
print(X_train[1])

[431, 425, 1, 1605, 14838, 113, 66, 2, 848, 13, 27, 14, 27, 15, 50725, 3, 431, 374, 16, 9, 67507, 6, 52258, 3, 42, 4009, 783, 325, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[15874, 1072, 854, 1310, 4250, 13, 27, 14, 27, 15, 929, 797, 320, 15874, 98, 3, 27657, 28, 5, 4459, 11, 564, 52790, 8, 80617, 2125, 7, 2, 525, 241, 3, 28, 3890, 82814, 6574, 10, 206, 359, 6, 2, 126, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Cool, maintenant on peut créer un `torch.tensor` avec toutes les phrases.

In [33]:
X_train_torch = torch.tensor(X_train)
Y_train_torch = torch.tensor(Y_train)

print(X_train_torch.shape)
print(Y_train_torch.shape)

torch.Size([120000, 207])
torch.Size([120000])


## Création du réseau de neurones
Notre réseau de neurones va prendre en entrée un `torch.tensor` de taille `(batch_size, max_sentence_length)`, et va retourner un `torch.tensor` de taille `(batch_size, 4)`, où chaque ligne correspond à la probabilité d'appartenance à chaque classe.
Le réseau contient une couche d'embedding, qui va transformer les index des mots en vecteurs de taille `embedding_dim` (ou `D` comme on a dit plus haut), et deux couche 2 linéaires.
On va devoir choisir la taille de nos vecteurs d'embedding `embedding_dim`.

In [ ]:
import torch.nn as nn

class Net(nn.Module):
	def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
		super().__init__()
		self.embedding = nn.Embedding(vocab_size, embedding_dim)
		self.fc1 = nn.Linear(embedding_dim, hidden_dim)
		self.fc2 = nn.Linear(hidden_dim, output_dim)

		self.relu = nn.ReLU()

	def forward(self, x):
		# x.shape = (batch_size, max_sentence_length)
		embedded = self.embedding(x)
		# embedded.shape = (batch_size, max_sentence_length, embedding_dim)
		embedded = embedded.mean(dim=1)
		# embedded.shape = (batch_size, embedding_dim)
		out = self.fc1(embedded)
		out = self.relu(out)
		out = self.fc2(out)
		return out

Comme vous pouvez voir dans le `forward`, on utilise la couche d'embedding, pour transformer les index des mots en vecteurs de taille `embedding_dim`, puis on calcule la moyenne sur la dimension des phrases pour avoir un vecteur de taille fixe `embedding_dim` pour chaque phrase afin de le passer dans les couches linéaires.

Mais on ne veut pas prendre en compte les vecteurs qu'on a ajouté pour le padding, donc on va devoir modifier le `forward` pour ne pas les prendre en compte.
Comment faire ça?
On va créer un `mask` qui va être un `torch.tensor` de taille `(batch_size, max_sentence_length)` qui vaut `True` si le mot est un mot du vocabulaire et `False` si le mot est un padding.
On va ensuite multiplier `embedded` par `mask.float()` pour mettre à 0 les vecteurs des mots qui sont des paddings.

Mais, il y a un problème, `mask` est de taille `(batch_size, max_sentence_length)` et `embedded` est de taille `(batch_size, max_sentence_length, embedding_dim)`, donc on ne peut pas les multiplier. On va donc devoir modifier `mask` pour qu'il soit de taille `(batch_size, max_sentence_length, embedding_dim)`.

Pour ça, on va utiliser la fonction `unsqueeze` (documentation: https://pytorch.org/docs/stable/generated/torch.unsqueeze.html) qui permet d'ajouter une dimension à un `torch.tensor`. Donc notre `mask` de taille `(batch_size, max_sentence_length)` va devenir de taille `(batch_size, max_sentence_length, 1)`.

Comme `torch` est intelligent, lorsque qu'on fait `embedded * mask.float()`, il va automatiquement répéter `mask` sur la dimension `embedding_dim` pour qu'il soit de taille `(batch_size, max_sentence_length, embedding_dim)`. Cette opération s'appelle le `broadcasting`.

Une fois qu'on a fait ça, on peut calculer la moyenne sur la dimension des phrases, et on obtient un `torch.tensor` de taille `(batch_size, embedding_dim)` qu'on peut passer dans les couches linéaires.

**Attention**: si on utilise `torch.mean`, il va diviser par la longueur maximale des phrases (donc il prend en compte les paddings), donc on va devoir utiliser `torch.sum` et diviser par le nombre de mots qui ne sont pas des paddings.

In [ ]:
import torch.nn as nn

class Net(nn.Module):
	def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
		super().__init__()
		self.embedding = nn.Embedding(vocab_size, embedding_dim)
		self.fc1 = nn.Linear(embedding_dim, hidden_dim)
		self.fc2 = nn.Linear(hidden_dim, output_dim)

		self.relu = nn.ReLU()

	def forward(self, x):
		# x.shape = (batch_size, max_sentence_length)

		embedded = self.embedding(x)
		# embedded.shape = (batch_size, max_sentence_length, embedding_dim)

		mask = (x != vocab["<pad>"])
		# mask.shape = (batch_size, max_sentence_length)

		...
		# mask.shape = (batch_size, max_sentence_length, 1)

		embedded = ... # multiplier embedded par mask.float() pour mettre à 0 les vecteurs des mots qui sont des paddings
		# embedded.shape = (batch_size, max_sentence_length, embedding_dim)

		# calculer la moyenne
		...

		out = self.fc1(embedded)
		out = self.relu(out)
		out = self.fc2(out)
		return out

## Entraînement
La boucle d'entraînement est la même que pour les TPs précédents, donc à vous de jouer! Utilisez la loss `nn.CrossEntropyLoss` et l'optimiseur `torch.optim.Adam`. N'oubliez pas de batcher les données et d'entraîner sur le gpu.

In [ ]:
...

## Évaluation sur l'accuracy

In [ ]:
...

## Amélioration possible
Vous pouvez remarquer que le tokenizer n'enlève pas les ponctuations, donc vous pouvez essayer de les enlever pour voir si ça améliore les performances.

## Challenge: sharing link very soon